## Where are the rail routes?

In [1]:
import _section1_utils as section1
import geopandas as gpd
import merge_data
import merge_operator_data
import numpy as np
import pandas as pd
from segment_speed_utils import gtfs_schedule_wrangling, helpers
from shared_utils import portfolio_utils
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS, SEGMENT_GCS

In [2]:
import sys

sys.path.append("../gtfs_funnel/")
import route_typologies

In [3]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [4]:
org_name_lists = ["Capitol Corridor Joint Powers Authority"]

In [5]:
organization_name = "Capitol Corridor Joint Powers Authority"

In [6]:
analysis_date_list = ["2024-11-13"]

In [7]:
analysis_date = "2024-11-13"

In [8]:
schd_keys = [
    "f5a749dd65924e025b1293c58f95f8d6",
]

In [9]:
name_list = ["Bay Area 511 Capitol Corridor Schedule"]

In [10]:
ROUTE_TYPOLOGIES = GTFS_DATA_DICT.schedule_tables.route_typologies
route_typologies_df = pd.read_parquet(
    f"{SCHED_GCS}{ROUTE_TYPOLOGIES}_{analysis_date}.parquet"
)

In [11]:
route_typologies_df.head(1)

,schedule_gtfs_dataset_key,name,route_type,route_id,route_long_name,route_short_name,combined_name,is_express,is_rapid,is_rail,is_local,direction_id,common_shape_id,route_name,route_meters,is_coverage,is_downtown_local
0,1770249a5a2e770ca90628434d4934b1,VCTC GMV Schedule,3,3407,Route 18,Route 18,Route 18__Route 18,0,1,0,0,0.00,18827,OHS TRIPPER - C ST - DORIS ST,8611.13,1,0


### Test Files

In [12]:
SCHED_GCS

'gs://calitp-analytics-data/data-analyses/gtfs_schedule/'

In [13]:
GTFS_DATA_DICT.schedule_tables.route_typologies

'nacto_typologies/route_typologies'

In [14]:
nov_test = pd.read_parquet("gs://calitp-analytics-data/data-analyses/gtfs_schedule/nacto_typologies/route_typologies_AH_TEST_2024-11-13.parquet")

In [15]:
dec_test = pd.read_parquet("gs://calitp-analytics-data/data-analyses/gtfs_schedule/nacto_typologies/route_typologies_AH_TEST_2024-12-11.parquet")

In [16]:
dec_test.head(2)

,schedule_gtfs_dataset_key,name,route_type,route_id,route_long_name,route_short_name,combined_name,is_express,is_rapid,is_rail,is_local,direction_id,common_shape_id,route_name,route_meters,is_coverage,is_downtown_local
0,1770249a5a2e770ca90628434d4934b1,VCTC GMV Schedule,3,3407,Route 18,Route 18,Route 18__Route 18,0,1,0,0,0.00,18827,OHS TRIPPER - C ST - DORIS ST,8611.13,1,0
1,1770249a5a2e770ca90628434d4934b1,VCTC GMV Schedule,3,3407,Route 18,Route 18,Route 18__Route 18,0,1,0,0,1.00,18828,OHS TRIPPER - C ST - DORIS ST,10686.60,1,0


In [17]:
dec_og = pd.read_parquet("gs://calitp-analytics-data/data-analyses/gtfs_schedule/nacto_typologies/route_typologies_2024-12-11.parquet")

In [18]:
dec_og.loc[dec_og.route_id == "Blue Line"]

,schedule_gtfs_dataset_key,name,route_type,route_id,route_long_name,route_short_name,combined_name,is_express,is_rapid,is_rail,is_local,direction_id,common_shape_id,route_name,route_meters,is_coverage,is_downtown_local
1830,fb467982dcc77a7f9199bebe709bb700,Bay Area 511 Santa Clara Transit Schedule,0,Blue Line,Baypointe - Santa Teresa,Blue Line,Blue Line__Baypointe - Santa Teresa,0,1,1,0,0.00,115504,Baypointe - Santa Teresa,26917.60,0,1
1831,fb467982dcc77a7f9199bebe709bb700,Bay Area 511 Santa Clara Transit Schedule,0,Blue Line,Baypointe - Santa Teresa,Blue Line,Blue Line__Baypointe - Santa Teresa,0,1,1,0,1.00,115514,Baypointe - Santa Teresa,27091.11,0,1


#### Some routes are repeated 4x. 

In [19]:
rail_routes_dec_test = dec_test.loc[dec_test.is_rail == 1]

In [20]:
rail_routes_dec_test.loc[rail_routes_dec_test.route_id == "Blue Line"]

,schedule_gtfs_dataset_key,name,route_type,route_id,route_long_name,route_short_name,combined_name,is_express,is_rapid,is_rail,is_local,direction_id,common_shape_id,route_name,route_meters,is_coverage,is_downtown_local
1875,fb467982dcc77a7f9199bebe709bb700,Bay Area 511 Santa Clara Transit Schedule,0,Blue Line,Baypointe - Santa Teresa,Blue Line,Blue Line__Baypointe - Santa Teresa,0,1,1,0,0.00,115504,Baypointe - Santa Teresa,26917.60,0,1
1876,fb467982dcc77a7f9199bebe709bb700,Bay Area 511 Santa Clara Transit Schedule,0,Blue Line,Baypointe - Santa Teresa,Blue Line,Blue Line__Baypointe - Santa Teresa,0,1,1,0,1.00,115514,Baypointe - Santa Teresa,27091.11,0,1


In [22]:
rail_routes_dec_test.loc[rail_routes_dec_test.route_id == "CC"]

,schedule_gtfs_dataset_key,name,route_type,route_id,route_long_name,route_short_name,combined_name,is_express,is_rapid,is_rail,is_local,direction_id,common_shape_id,route_name,route_meters,is_coverage,is_downtown_local
1534,f5a749dd65924e025b1293c58f95f8d6,Bay Area 511 Capitol Corridor Schedule,2,CC,Capitol Corridor,CC,CC__Capitol Corridor,0,1,1,1,0.00,5cf6811a-2f53-4199-b315-4408eb816e82,"Daily train service between Auburn, Sacramento, Oakland and San Jose",143715.40,0,0
1535,f5a749dd65924e025b1293c58f95f8d6,Bay Area 511 Capitol Corridor Schedule,2,CC,Capitol Corridor,CC,CC__Capitol Corridor,0,0,1,0,1.00,92232e9e-55e8-4f8a-bb86-1aeb515be83e,"Daily train service between Auburn, Sacramento, Oakland and San Jose",143684.90,1,0


In [21]:
rail_routes_dec_test.route_id.value_counts().describe()

count   56.00
mean     1.77
std      0.43
min      1.00
25%      2.00
50%      2.00
75%      2.00
max      2.00
Name: route_id, dtype: float64

In [ ]:
rail_routes_dec_test2 = rail_routes_dec_test.drop_duplicates(subset = ["schedule_gtfs_dataset_key", "route_id", "route_long_name", "direction_id"])

In [ ]:
rail_routes_dec_test2.route_id.value_counts().describe()

In [ ]:
rail_routes_dec_test2.loc[rail_routes_dec_test2.route_id == "Blue Line"]

In [ ]:
# rail_routes_dec_test2.groupby(['route_id']).agg({'route_long_name':'count'})

### Fix `operator_profile`
* Nothing shows up for `# Rail Route Types` even though there's one rail route.

In [ ]:
operator_profiles = section1.load_operator_ntd_profile(organization_name)

In [ ]:
operator_profiles

In [ ]:
f"{GTFS_DATA_DICT.digest_tables.dir}{GTFS_DATA_DICT.digest_tables.operator_profiles}.parquet"

#### Go to `gtfs_digest/merge_operator_data/concatenate_rt_vs_schedule_operator_metrics`

In [ ]:
op_rt_sched_metrics = merge_operator_data.concatenate_rt_vs_schedule_operator_metrics(
    analysis_date_list
)

In [ ]:
op_rt_sched_metrics.head(1)

In [ ]:
op_rt_sched_metrics = op_rt_sched_metrics.loc[
    op_rt_sched_metrics.schedule_gtfs_dataset_key.isin(schd_keys)
]

In [ ]:
op_rt_sched_metrics

#### File I need to correct is in `data-analyses/gtfs_funnel/operator_scheduled_stats.py`

In [ ]:
GTFS_DATA_DICT.schedule_tables.operator_scheduled_stats

In [ ]:
ROUTE_TYPOLOGY = GTFS_DATA_DICT.schedule_tables.route_typologies

#### I thought I ran this yesterday

In [ ]:
route_typology = pd.read_parquet(f"{SCHED_GCS}{ROUTE_TYPOLOGY}_{analysis_date}.parquet")

In [ ]:
analysis_date

In [ ]:
route_typology = route_typology.loc[route_typology.name.isin(name_list)]

In [ ]:
route_typology

### Only one route appears.

### Check out `gtfs_funnel/route_typologies/categorize_routes_by_name`
* 3 routes appear. One is rail.
* All 3 routes are retained until the end.

In [ ]:
df = helpers.import_scheduled_trips(
    analysis_date,
    columns=[
        "gtfs_dataset_key",
        "name",
        "route_type",
        "route_id",
        "route_long_name",
        "route_short_name",
    ],
    get_pandas=True,
)

In [ ]:
df = df.loc[df.name.isin(name_list)]

In [ ]:
df = df.assign(
    route_id=df.route_id.fillna(""),
    route_short_name=df.route_short_name.fillna(""),
    route_long_name=df.route_long_name.fillna(""),
)

In [ ]:
df = df.assign(combined_name=df.route_short_name + "__" + df.route_long_name)

In [ ]:
df

In [ ]:
def tag_rapid_express_rail(route_name_string: str, route_type_string: str) -> pd.Series:
    """
    Use the combined route_name and see if we can
    tag out words that indicate the route is
    express, rapid, local, and rail.

    Treat rail as own category.
    For local routes, we'll pass that through NACTO to see
    if we can better categorize as downtown_local, local, or coverage.
    """
    route_name_string = route_name_string.lower()

    express = 0
    rapid = 0
    rail = 0

    if any(substring in route_name_string for substring in ["express", "limited"]):
        express = 1
    if "rapid" in route_name_string:
        rapid = 1

    rail_types = ["0", "1", "2", "5", "6", "7", "11", "12"]
    if route_type_string in rail_types:
        rail = 1

    return pd.Series(
        [express, rapid, rail], index=["is_express", "is_rapid", "is_rail"]
    )

I think you'd want to separate rail from the other types before the filtering.
Rail is always tagged here.
The other types that run on roads can have multiple types per route (a bus that runs partway in downtown, partway on local roads, etc). The filtering for typology gets at what might be the most common typologies present (over 10% of the route), and then allows 2 typologies to remain...primary/secondary typology by the end of the script. You can take a look at the relative proportions...but sometimes there's no clear plurality. Feel free to adjust the thresholds for what makes sense, 10% is fairly loose, but you might find dropping it to 5% leaves you with too many rows per route-dir. Find the balance!
The intermediate output produced in this step is: GTFS_DATA_DICT.schedule_tables.route_typologies , and you would be able to control how you want that column to be flagged as an independent table before you merge to GTFS digest without rerunning everything.

In [ ]:
typology_tags = df.apply(
    lambda x: tag_rapid_express_rail(x.combined_name, x.route_type), axis=1
)

In [ ]:
typology_tags.shape

In [ ]:
typology_tags

In [ ]:
df2 = pd.concat([df, typology_tags], axis=1)

In [ ]:
df2

In [ ]:
df2 = df2.assign(
    is_local=df2.apply(
        lambda x: (
            1 if (x.is_express == 0) and (x.is_rapid == 0) and (x.is_rail == 0) else 0
        ),
        axis=1,
    ).astype(int)
)

In [ ]:
df2.columns

### `prep_roads`

In [ ]:
roads = route_typologies.prep_roads(GTFS_DATA_DICT)

### `overlay_shapes_to_roads`
* The other 2 routes disappear here. 
* Break it out here. 

In [ ]:
ROAD_BUFFER_METERS = 20

#### `common_shape` doesn't have the rail routes -> go to `segment_speed_utils/gtfs_schedule_wrangling` to find it.

In [ ]:
route_dir_cols = ["gtfs_dataset_key", "route_id", "direction_id"]

keep_trip_cols = route_dir_cols + ["trip_instance_key", "shape_id", "shape_array_key"]

trips = helpers.import_scheduled_trips(
    analysis_date, columns=keep_trip_cols, get_pandas=True
).rename(columns={"schedule_gtfs_dataset_key": "gtfs_dataset_key"})
sorting_order = [True for i in route_dir_cols]

In [ ]:
trips = trips.loc[trips.gtfs_dataset_key.isin(schd_keys)]

#### `CC` is the rail.

In [ ]:
trips[["route_id"]].drop_duplicates()

In [ ]:
most_common_shape = (
    trips.groupby(
        route_dir_cols + ["shape_id", "shape_array_key"],
        observed=True,
        group_keys=False,
        dropna=False,
    )
    .agg({"trip_instance_key": "count"})
    .reset_index()
    .sort_values(
        route_dir_cols + ["trip_instance_key"], ascending=sorting_order + [False]
    )
    .drop_duplicates(subset=route_dir_cols)
    .reset_index(drop=True)[route_dir_cols + ["shape_id", "shape_array_key"]]
).rename(
    columns={
        "gtfs_dataset_key": "schedule_gtfs_dataset_key",
        "shape_id": "common_shape_id",
    }
)

In [ ]:
most_common_shape[["route_id"]].drop_duplicates()

In [ ]:
shape_geom = helpers.import_scheduled_shapes(
    analysis_date,
    columns=["shape_array_key", "geometry"],
)

In [ ]:
common_shape_geom = pd.merge(
    shape_geom, most_common_shape, on="shape_array_key", how="inner"
).drop(columns="shape_array_key")

In [ ]:
common_shape_geom.drop(columns=["geometry"])

In [ ]:
route_info = (
    helpers.import_scheduled_trips(
        analysis_date,
        columns=[
            "gtfs_dataset_key",
            "route_id",
            "route_long_name",
            "route_short_name",
            "route_desc",
        ],
    )
    .drop_duplicates()
    .pipe(portfolio_utils.add_route_name)
    .drop(columns=["route_long_name", "route_short_name", "route_desc"])
)

In [ ]:
route_info.columns

In [ ]:
route_info = route_info.loc[route_info.schedule_gtfs_dataset_key.isin(schd_keys)]

In [ ]:
route_info

In [ ]:
common_shape_geom2 = pd.merge(
    common_shape_geom,
    route_info.rename(columns={"route_name_used": "route_name"}),
    on=["schedule_gtfs_dataset_key", "route_id"],
)

In [ ]:
common_shape_geom2[["route_id"]].drop_duplicates()

In [ ]:
common_shape = gtfs_schedule_wrangling.most_common_shape_by_route_direction(
    analysis_date
)

In [ ]:
common_shape.columns

In [ ]:
common_shape = common_shape.loc[common_shape.schedule_gtfs_dataset_key.isin(schd_keys)]

In [ ]:
common_shape[["route_id", "direction_id"]].drop_duplicates()

In [ ]:
# common_shape.explore("route_id")

In [ ]:
common_shape_piped = gtfs_schedule_wrangling.most_common_shape_by_route_direction(
    analysis_date
).pipe(helpers.remove_shapes_outside_ca)

In [ ]:
common_shape_piped = common_shape_piped.loc[
    common_shape_piped.schedule_gtfs_dataset_key.isin(schd_keys)
]

In [ ]:
common_shape_piped[["route_id"]].drop_duplicates()

#### Continuing with `gtfs_funnel/route_typologies` the `if==main`

In [ ]:
ROAD_BUFFER_METERS = 20

#### Tweak the threshold lightly to 0.09

In [ ]:
TYPOLOGY_THRESHOLD = 0.09

In [ ]:
gdf = route_typologies.overlay_shapes_to_roads(roads, analysis_date, ROAD_BUFFER_METERS)

In [ ]:
gdf.columns

In [ ]:
gdf = gdf.loc[gdf.schedule_gtfs_dataset_key.isin(schd_keys)]

In [ ]:
gdf.route_id.nunique()

In [ ]:
# gdf[["route_id", "direction_id", "typology", "pct_typology"]].drop_duplicates().sort_values(
#    by=["route_id","direction_id", "pct_typology"]
# )

#### Filtering out for typology_threshold throws away the other routes -> Find a way to retain rail routes. 

In [ ]:
route_typology_df = gdf.loc[(gdf.pct_typology >= TYPOLOGY_THRESHOLD)]

In [ ]:
route_typology_df

#### Getting rail routes back

In [ ]:
def add_rail_back(
    categorize_routes_df: pd.DataFrame, overlay_shapes_to_roads_df: pd.DataFrame
) -> pd.DataFrame:
    """
    categorize_routes_df: df created by categorize_routes_by_name()
    overlay_shapes_to_roads_df: df created by overlay_shapes_to_roads() 
    """
    # Filter out for only rail routes and drop duplicates.
    rail_routes = categorize_routes_df.loc[categorize_routes_df.is_rail == 1][
        ["route_id", "schedule_gtfs_dataset_key"]
    ].drop_duplicates()

    # Merge with route_typologies_df to retain the details for
    # columns such as typology, freq_category, etc
    m1 = pd.merge(gdf, rail_routes, how="inner")

    # Retain only one row for each route-direction-operator
    # keeping the row with the highest pct_typology
    m1 = m1.sort_values(
        by=["route_id", "direction_id", "schedule_gtfs_dataset_key", "pct_typology"],
        ascending=[True, True, True, False],
    ).drop_duplicates(subset=["route_id", "direction_id", "schedule_gtfs_dataset_key"])
    
    # Apply primary_secondary_typology() function which adds
    # columns like is_nacto_rapid, is_nacto_coverage
    m1 = route_typologies.primary_secondary_typology(m1)

    return m1

In [ ]:
rail_routes_df = add_rail_back(df2, gdf)

In [ ]:
rail_routes_df

#### Back to the rest of the function without any change. 

In [ ]:
route_typology_df.head(2)

In [ ]:
route_typology_df2 = route_typologies.primary_secondary_typology(route_typology_df)

In [ ]:
route_typology_df2.shape

In [ ]:
route_typology_df2

In [ ]:
route_tagged = route_typologies.categorize_routes_by_name(analysis_date)

In [ ]:
route_tagged = route_tagged.loc[route_tagged.schedule_gtfs_dataset_key.isin(schd_keys)]

In [ ]:
route_tagged

#### Easier way to keep rail routes = use an `outer` join instead of a `left` join.
* But there are a lot of missing values.

In [ ]:
df3 = pd.merge(
    route_tagged,
    route_typology_df2,
    on=["schedule_gtfs_dataset_key", "route_id"],
    how="outer",
)

In [ ]:
df3

#### Incorporating my `rail_routes_df2`

In [ ]:
#  First concat it with route_typology_df2
all_routes = pd.concat([route_typology_df2, rail_routes_df])

In [ ]:
# merge with route_tagged as normal
df4 = pd.merge(
    route_tagged, all_routes, on=["schedule_gtfs_dataset_key", "route_id"], how="inner"
)

In [ ]:
df4